In [10]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
azure_openai_api_key = os.getenv('AZURE_OPENAI_API_KEY_4')
azure_openai_endpoint = os.getenv('AZURE_OPENAI_API_ENDPOINT_4')
deployment_name = os.getenv('AZURE_DEPLOYMENT_NAME_4')


# Set up Azure OpenAI API (for Azure deployment)
openai.api_type = "azure"
openai.api_key = azure_openai_api_key
openai.api_base = azure_openai_endpoint
openai.api_version = "2023-05-15"  # Ensure this is up to date

In [4]:
import pandas as pd
df = pd.read_csv("gutenberg_ebooks.csv")
df

,Ebook ID,Author,Title,Credits,Summary,Language,LoC Class,Subject,Subject_2,Subject_3,Subject_4,Category,EBook-No.,Release Date,Most Recently Updated,Copyright Status,Downloads
0,8600,"Zola, Émile, 1840-1902",L'Assommoir,"John Bickers, Dagny and David Widger","""L'Assommoir"" by Émile Zola is a novel written...",English,PQ: Language and Literatures: Romance literatu...,Domestic fiction,Married women -- Fiction,Paris (France) -- Fiction,Working class women -- Fiction,Text,8600,"Apr 27, 2006","Sep 15, 2022",Public domain in the USA.,455 downloads in the last 30 days.
1,8601,"Tennyson, Alfred Tennyson, Baron, 1809-1892",The Early Poems of Alfred Lord Tennyson,"Produced by Jonathan Ingram, Clytie Siddall, C...","""The Early Poems of Alfred Lord Tennyson"" by J...",English,PR: Language and Literatures: English literature,English poetry -- 19th century,NaN,NaN,NaN,Text,8601,"Aug 1, 2005","Jun 21, 2020",Public domain in the USA.,603 downloads in the last 30 days.
2,8602,"Riddell, J. H., Mrs., 1832-1906",The Uninhabited House,"Produced by Suzanne Shell, Martin Agren, Charl...","""The Uninhabited House"" by Mrs. J. H. Riddell ...",English,PR: Language and Literatures: English literature,Ghost stories,English fiction -- 19th century,NaN,NaN,Text,8602,"Aug 1, 2005","Apr 10, 2014",Public domain in the USA.,90 downloads in the last 30 days.
3,8603,Various,"Notes and Queries, Number 01, November 3, 1849",Produced by Internet Library of Early Journals...,"""Notes and Queries, Number 01, November 3, 184...",English,AG: General Works: Dictionaries and other gene...,Questions and answers -- Periodicals,NaN,NaN,NaN,Text,8603,"Aug 1, 2005","Jan 17, 2013",Public domain in the USA.,50 downloads in the last 30 days.
4,8604,"Aeschylus, 526 BCE-457 BCE","The House of Atreus; Being the Agamemnon, the ...","Ted Garvin, Lorna Hanrahan, Charles Franks, an...","""The House of Atreus; Being the Agamemnon, the...",English,PA: Language and Literatures: Classical Langua...,"Agamemnon, King of Mycenae (Mythological chara...","Orestes, King of Argos (Mythological character...",NaN,NaN,Text,8604,"Aug 1, 2005","Jun 19, 2023",Public domain in the USA.,210 downloads in the last 30 days.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19924,19996,Various,"The Atlantic Monthly, Volume 16, No. 96, Octob...","Produced by Joshua Hutchinson, Josephine Paolu...","""The Atlantic Monthly, Volume 16, No. 96, Octo...",English,AP: General Works: Periodicals,American periodicals,NaN,NaN,NaN,Text,19996,"Dec 2, 2006",NaN,Public domain in the USA.,88 downloads in the last 30 days.
19925,19997,"Ober, Frederick A. (Frederick Albion), 1849-1913",Amerigo Vespucci,"Produced by Suzanne Shell, Richard J. Shiffer ...","""Amerigo Vespucci"" by Frederick A. Ober is a h...",English,E011: History: America: America,"Vespucci, Amerigo, 1451-1512",NaN,NaN,NaN,Text,19997,"Dec 3, 2006",NaN,Public domain in the USA.,145 downloads in the last 30 days.
19926,19998,"Allen, Lewis Falley, 1800-1890",Rural ArchitectureBeing a Complete Description...,"Produced by Louise Hope, Steven Giacomelli and...","""Rural Architecture"" by Lewis Falley Allen is ...",English,NA: Fine Arts: Architecture,Domestic animals,"Architecture, Domestic",NaN,NaN,Text,19998,"Dec 3, 2006",NaN,Public domain in the USA.,232 downloads in the last 30 days.
19927,19999,"Trowbridge, J. T. (John Townsend), 1827-1916",The Drummer Boy,Produced by David Edwards and the Online Distr...,"""The Drummer Boy"" by J. T. Trowbridge is a his...",English,PZ: Language and Literatures: Juvenile belles ...,"United States -- History -- Civil War, 1861-18...",NaN,NaN,NaN,Text,19999,"Dec 3, 2006",NaN,Public domain in the USA.,76 downloads in the last 30 days.


In [5]:
import pandas as pd

# only removes funny tokens for English texts

def remove_funny_tokens(text):
    tokens = text.split()
    sample = ' '.join(' '.join(tokens).replace('xe2x80x9c', ' ').replace('xe2x80x9d', ' ') \
                                      .replace('xe2x80x94', ' ').replace('xe2x80x99', "'") \
                                      .replace('xe2x80x98', "'").split())
    return sample
# clean newlines, carriage returns and tabs
def clean_text(text):
    if not isinstance(text, str):  # Vérifiez si le texte est une chaîne
        return text  # Retournez la valeur d'origine si ce n'est pas une chaîne
    
    cleaned_listed_text = []
    listed_text = list(text)

    for iter in range(len(listed_text) - 1):
        if (listed_text[iter] == '\\' and listed_text[iter + 1] == 'n') or \
            (listed_text[iter] == 'n' and listed_text[iter - 1] == '\\'):
            continue
        elif listed_text[iter] == '\\' and listed_text[iter + 1] == 'r' or \
            (listed_text[iter] == 'r' and listed_text[iter - 1] == '\\'):
            continue
        elif listed_text[iter] == '\\' and listed_text[iter + 1] == 't' or \
            (listed_text[iter] == 't' and listed_text[iter - 1] == '\\'):
            continue
        elif listed_text[iter] == '\\':
            continue
        else:
            cleaned_listed_text.append(listed_text[iter])

    cleaned_text = ''.join([str(char) for char in cleaned_listed_text])
    cleaned_text = remove_funny_tokens(cleaned_text)

    return ''.join(cleaned_text)

# Charger le CSV déjà existant
df_data = pd.read_csv('gutenberg_ebooks.csv')

# Nettoyer le texte dans la colonne 'Summary'
df_data['Summary'] = df_data['Summary'].apply(clean_text)

# Sauvegarder les données nettoyées dans un nouveau CSV
df_data.to_csv('gutenberg_cleaned_ebooks.csv', index=False)


In [6]:

df = pd.read_csv("gutenberg_cleaned_ebooks.csv")
df

,Ebook ID,Author,Title,Credits,Summary,Language,LoC Class,Subject,Subject_2,Subject_3,Subject_4,Category,EBook-No.,Release Date,Most Recently Updated,Copyright Status,Downloads
0,8600,"Zola, Émile, 1840-1902",L'Assommoir,"John Bickers, Dagny and David Widger","""L'Assommoir"" by Émile Zola is a novel written...",English,PQ: Language and Literatures: Romance literatu...,Domestic fiction,Married women -- Fiction,Paris (France) -- Fiction,Working class women -- Fiction,Text,8600,"Apr 27, 2006","Sep 15, 2022",Public domain in the USA.,455 downloads in the last 30 days.
1,8601,"Tennyson, Alfred Tennyson, Baron, 1809-1892",The Early Poems of Alfred Lord Tennyson,"Produced by Jonathan Ingram, Clytie Siddall, C...","""The Early Poems of Alfred Lord Tennyson"" by J...",English,PR: Language and Literatures: English literature,English poetry -- 19th century,NaN,NaN,NaN,Text,8601,"Aug 1, 2005","Jun 21, 2020",Public domain in the USA.,603 downloads in the last 30 days.
2,8602,"Riddell, J. H., Mrs., 1832-1906",The Uninhabited House,"Produced by Suzanne Shell, Martin Agren, Charl...","""The Uninhabited House"" by Mrs. J. H. Riddell ...",English,PR: Language and Literatures: English literature,Ghost stories,English fiction -- 19th century,NaN,NaN,Text,8602,"Aug 1, 2005","Apr 10, 2014",Public domain in the USA.,90 downloads in the last 30 days.
3,8603,Various,"Notes and Queries, Number 01, November 3, 1849",Produced by Internet Library of Early Journals...,"""Notes and Queries, Number 01, November 3, 184...",English,AG: General Works: Dictionaries and other gene...,Questions and answers -- Periodicals,NaN,NaN,NaN,Text,8603,"Aug 1, 2005","Jan 17, 2013",Public domain in the USA.,50 downloads in the last 30 days.
4,8604,"Aeschylus, 526 BCE-457 BCE","The House of Atreus; Being the Agamemnon, the ...","Ted Garvin, Lorna Hanrahan, Charles Franks, an...","""The House of Atreus; Being the Agamemnon, the...",English,PA: Language and Literatures: Classical Langua...,"Agamemnon, King of Mycenae (Mythological chara...","Orestes, King of Argos (Mythological character...",NaN,NaN,Text,8604,"Aug 1, 2005","Jun 19, 2023",Public domain in the USA.,210 downloads in the last 30 days.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19924,19996,Various,"The Atlantic Monthly, Volume 16, No. 96, Octob...","Produced by Joshua Hutchinson, Josephine Paolu...","""The Atlantic Monthly, Volume 16, No. 96, Octo...",English,AP: General Works: Periodicals,American periodicals,NaN,NaN,NaN,Text,19996,"Dec 2, 2006",NaN,Public domain in the USA.,88 downloads in the last 30 days.
19925,19997,"Ober, Frederick A. (Frederick Albion), 1849-1913",Amerigo Vespucci,"Produced by Suzanne Shell, Richard J. Shiffer ...","""Amerigo Vespucci"" by Frederick A. Ober is a h...",English,E011: History: America: America,"Vespucci, Amerigo, 1451-1512",NaN,NaN,NaN,Text,19997,"Dec 3, 2006",NaN,Public domain in the USA.,145 downloads in the last 30 days.
19926,19998,"Allen, Lewis Falley, 1800-1890",Rural ArchitectureBeing a Complete Description...,"Produced by Louise Hope, Steven Giacomelli and...","""Rural Architecture"" by Lewis Falley Allen is ...",English,NA: Fine Arts: Architecture,Domestic animals,"Architecture, Domestic",NaN,NaN,Text,19998,"Dec 3, 2006",NaN,Public domain in the USA.,232 downloads in the last 30 days.
19927,19999,"Trowbridge, J. T. (John Townsend), 1827-1916",The Drummer Boy,Produced by David Edwards and the Online Distr...,"""The Drummer Boy"" by J. T. Trowbridge is a his...",English,PZ: Language and Literatures: Juvenile belles ...,"United States -- History -- Civil War, 1861-18...",NaN,NaN,NaN,Text,19999,"Dec 3, 2006",NaN,Public domain in the USA.,76 downloads in the last 30 days.


In [7]:
# Sélection des colonnes souhaitées
colonnes_souhaitees = ['Ebook ID', 'Author', 'Title', 'Credits', 'Summary', 'Language', 'LoC Class', 'Subject']
df_filtre = df[colonnes_souhaitees]

# Affichage du DataFrame filtré
df_filtre

,Ebook ID,Author,Title,Credits,Summary,Language,LoC Class,Subject
0,8600,"Zola, Émile, 1840-1902",L'Assommoir,"John Bickers, Dagny and David Widger","""L'Assommoir"" by Émile Zola is a novel written...",English,PQ: Language and Literatures: Romance literatu...,Domestic fiction
1,8601,"Tennyson, Alfred Tennyson, Baron, 1809-1892",The Early Poems of Alfred Lord Tennyson,"Produced by Jonathan Ingram, Clytie Siddall, C...","""The Early Poems of Alfred Lord Tennyson"" by J...",English,PR: Language and Literatures: English literature,English poetry -- 19th century
2,8602,"Riddell, J. H., Mrs., 1832-1906",The Uninhabited House,"Produced by Suzanne Shell, Martin Agren, Charl...","""The Uninhabited House"" by Mrs. J. H. Riddell ...",English,PR: Language and Literatures: English literature,Ghost stories
3,8603,Various,"Notes and Queries, Number 01, November 3, 1849",Produced by Internet Library of Early Journals...,"""Notes and Queries, Number 01, November 3, 184...",English,AG: General Works: Dictionaries and other gene...,Questions and answers -- Periodicals
4,8604,"Aeschylus, 526 BCE-457 BCE","The House of Atreus; Being the Agamemnon, the ...","Ted Garvin, Lorna Hanrahan, Charles Franks, an...","""The House of Atreus; Being the Agamemnon, the...",English,PA: Language and Literatures: Classical Langua...,"Agamemnon, King of Mycenae (Mythological chara..."
...,...,...,...,...,...,...,...,...
19924,19996,Various,"The Atlantic Monthly, Volume 16, No. 96, Octob...","Produced by Joshua Hutchinson, Josephine Paolu...","""The Atlantic Monthly, Volume 16, No. 96, Octo...",English,AP: General Works: Periodicals,American periodicals
19925,19997,"Ober, Frederick A. (Frederick Albion), 1849-1913",Amerigo Vespucci,"Produced by Suzanne Shell, Richard J. Shiffer ...","""Amerigo Vespucci"" by Frederick A. Ober is a h...",English,E011: History: America: America,"Vespucci, Amerigo, 1451-1512"
19926,19998,"Allen, Lewis Falley, 1800-1890",Rural ArchitectureBeing a Complete Description...,"Produced by Louise Hope, Steven Giacomelli and...","""Rural Architecture"" by Lewis Falley Allen is ...",English,NA: Fine Arts: Architecture,Domestic animals
19927,19999,"Trowbridge, J. T. (John Townsend), 1827-1916",The Drummer Boy,Produced by David Edwards and the Online Distr...,"""The Drummer Boy"" by J. T. Trowbridge is a his...",English,PZ: Language and Literatures: Juvenile belles ...,"United States -- History -- Civil War, 1861-18..."


In [16]:
import pandas as pd
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage


# Charger le CSV existant
df_data = pd.read_csv('gutenberg_cleaned_ebooks.csv')

# Initialiser le modèle ChatOpenAI
chat_model = ChatOpenAI(model="gpt-3.5-turbo", 
                                openai_api_key=azure_openai_api_key,
                                engine=deployment_name)

# Fonction pour répondre aux questions
def ask_book_question(question, book_attributes):
    # Concaténer les attributs du livre pour fournir le contexte
    context = "\n".join(book_attributes)
    
    # Créer le message pour le modèle
    messages = [
        HumanMessage(content=f"{context}\n\nQuestion: {question}")
    ]
    
    # Utiliser le modèle pour générer la réponse
    response = chat_model(messages)
    return response.content

# Exemple d'utilisation
book_id = 1  # ID de l'ebook, à adapter selon votre dataset
book_info = df_filtre.iloc[book_id]  # Obtenir les informations du livre

# Attributs du livre à utiliser comme contexte
book_attributes = [
    f"Titre: {book_info['Title']}",
    f"Auteur: {book_info['Author']}",
    f"Langue: {book_info['Language']}",
    f"Sujet: {book_info['Subject']}",
    f"Résumé: {book_info['Summary']}",
]

# Poser une question
question = "De quels sujets traite le livre L'Assommoir?"
answer = ask_book_question(question, book_attributes)

print(f"Réponse: {answer}")


WARNING! engine is not default parameter.
                    engine was transferred to model_kwargs.
                    Please confirm that engine is what you intended.


Réponse: "L'Assommoir" est un roman d'Émile Zola, publié en 1877, faisant partie de la série des Rougon-Macquart. Il explore plusieurs sujets, notamment :

1. **L'alcoolisme** : Le roman décrit les effets destructeurs de l'alcool sur les individus et les familles, en se concentrant sur le personnage principal, Gervaise Macquart.

2. **La classe ouvrière** : Zola offre un portrait détaillé de la vie des ouvriers à Paris, mettant en lumière les difficultés économiques et sociales qu'ils rencontrent.

3. **La dégradation sociale** : Le livre montre la chute progressive de Gervaise et de sa famille, soulignant les forces sociales et personnelles qui contribuent à leur déclin.

4. **Le déterminisme social** : Fidèle à la philosophie naturaliste de Zola, le roman examine comment l'hérédité et le milieu social influencent le destin des personnages.

5. **Les relations familiales** : Les interactions au sein de la famille et leur détérioration sont un thème central, illustrant les tensions et 

In [20]:
# Filtrer les lignes où le titre est "L'Assommoir" et extraire la colonne Summary
summary_value = df_filtre.loc[df_filtre['Title'] == "L'Assommoir", 'Summary']

# Afficher la valeur
for value in summary_value:
    print(value)

"L'Assommoir" by Émile Zola is a novel written during the late 19th century, an era characterized by the realism movement in literature. The book explores the struggles of Gervaise, a laundress trying to build a life for herself and her children amidst the oppressive and often brutal conditions of working-class Paris. The story highlights themes of poverty, domestic strife, and the impact of alcoholism on individuals and families. The opening of "L'Assommoir" presents Gervaise in a state of despair, anxiously awaiting the return of Lantier, her partner, who has been increasingly unfaithful and irresponsible. The narrative depicts her emotional turmoil as she reflects on their meager living conditions and the struggles of single motherhood. Gervaise's observations of the bustling street below reveal the harsh realities faced by workers in the city, and through her interactions with Lantier, the text illustrates the complexities of love, frustration, and survival in a difficult social en

Créer un agent langchain capable de :
* répondre à des questions sur les livres et leurs attributs (exemple : de quels sujets traite tel ou tel livre ?)
* récupérer une liste des noms des personnages mentionnés dans le résumé du livre

In [24]:
import pandas as pd
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage



# Initialiser le modèle ChatOpenAI
chat_model = ChatOpenAI(model="gpt-3.5-turbo", 
                         openai_api_key=azure_openai_api_key,
                         engine=deployment_name)

# Fonction pour répondre aux questions
def ask_book_question(question, book_attributes):
    # Concaténer les attributs du livre pour fournir le contexte
    context = "\n".join(book_attributes)
    
    # Créer le message pour le modèle
    messages = [
        HumanMessage(content=f"{context}\n\nQuestion: {question}")
    ]
    
    # Utiliser le modèle pour générer la réponse
    response = chat_model(messages)
    return response.content

# Demander à l'utilisateur de saisir le titre du livre
book_title = input("Veuillez entrer le titre du livre : ")

# Filtrer les informations du livre correspondant au titre donné
book_info = df_filtre[df_filtre['Title'].str.contains(book_title, case=False, na=False)]

# Vérifier si le livre existe dans le DataFrame
if not book_info.empty:
    # Attributs du livre à utiliser comme contexte
    book_attributes = [
        f"Titre: {book_info.iloc[0]['Title']}",
        f"Auteur: {book_info.iloc[0]['Author']}",
        f"Langue: {book_info.iloc[0]['Language']}",
        f"Sujet: {book_info.iloc[0]['Subject']}",
        f"Résumé: {book_info.iloc[0]['Summary']}",
    ]

    # Poser une question pour extraire les noms des personnages, incluant le titre du livre
    question = f"Quels sont les noms des personnages mentionnés dans le résumé du livre '{book_info.iloc[0]['Title']}' ?"
    character_names = ask_book_question(question, book_attributes)
    

    # Afficher les noms des personnages
    print(f"Noms des personnages mentionnés dans le résumé : {character_names}")
else:
    print("Aucun livre trouvé avec ce titre.")


WARNING! engine is not default parameter.
                    engine was transferred to model_kwargs.
                    Please confirm that engine is what you intended.


Noms des personnages mentionnés dans le résumé : Les personnages mentionnés dans le résumé du livre "The Early Poems of Alfred Lord Tennyson" sont "Claribel" et "Mariana".


In [25]:
import pandas as pd
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage



# Initialiser le modèle ChatOpenAI
chat_model = ChatOpenAI(model="gpt-3.5-turbo", 
                         openai_api_key=azure_openai_api_key,
                         engine=deployment_name)

# Fonction pour répondre aux questions
def ask_book_question(question, book_attributes):
    # Concaténer les attributs du livre pour fournir le contexte
    context = "\n".join(book_attributes)
    
    # Créer le message pour le modèle
    messages = [
        HumanMessage(content=f"{context}\n\nQuestion: {question}")
    ]
    
    # Utiliser le modèle pour générer la réponse
    response = chat_model(messages)
    return response.content

# Demander à l'utilisateur de saisir le titre du livre
book_title = input("Veuillez entrer le titre du livre : ")

# Filtrer les informations du livre correspondant au titre donné
book_info = df_filtre[df_filtre['Title'].str.contains(book_title, case=False, na=False)]

# Vérifier si le livre existe dans le DataFrame
if not book_info.empty:
    # Attributs du livre à utiliser comme contexte
    book_attributes = [
        f"Titre: {book_info.iloc[0]['Title']}",
        f"Auteur: {book_info.iloc[0]['Author']}",
        f"Langue: {book_info.iloc[0]['Language']}",
        f"Sujet: {book_info.iloc[0]['Subject']}",
        f"Résumé: {book_info.iloc[0]['Summary']}",
    ]

    # Poser une question sur le sujet du livre, incluant le titre du livre
    question_subject = f"De quels sujets traite le livre '{book_info.iloc[0]['Title']}' ?"
    subjects = ask_book_question(question_subject, book_attributes)

    # Afficher les sujets
    print(f"Sujets traités dans le livre : {subjects}")

    # Poser une question pour extraire les noms des personnages
    question_characters = f"Quels sont les noms des personnages mentionnés dans le résumé du livre '{book_info.iloc[0]['Title']}' ?"
    character_names = ask_book_question(question_characters, book_attributes)

    # Afficher les noms des personnages
    print(f"Noms des personnages mentionnés dans le résumé : {character_names}")
else:
    print("Aucun livre trouvé avec ce titre.")


WARNING! engine is not default parameter.
                    engine was transferred to model_kwargs.
                    Please confirm that engine is what you intended.


Sujets traités dans le livre : Le livre "L'Assommoir" traite de plusieurs sujets centraux :

1. **Pauvreté** : Il explore les conditions de vie difficiles de la classe ouvrière à Paris.
2. **Alcoolisme** : Le roman met en lumière les effets destructeurs de l'alcool sur les individus et les familles.
3. **Conflit domestique** : Les tensions et les luttes au sein des relations familiales et de couple sont au cœur du récit.
4. **Espoir et survie** : Malgré les difficultés, Gervaise cherche à bâtir une vie meilleure pour elle-même et ses enfants.
5. **Réalisme social** : Zola décrit de manière détaillée et réaliste le quotidien des travailleurs et les défis qu'ils affrontent. 

Ces thèmes s'entrelacent pour dépeindre une image poignante et authentique de la vie ouvrière au XIXe siècle.
Noms des personnages mentionnés dans le résumé : Les personnages mentionnés dans le résumé du livre "L'Assommoir" sont Gervaise et Lantier.


In [ ]:
import pandas as pd
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage
import requests

# Initialiser le modèle ChatOpenAI
chat_model = ChatOpenAI(model="gpt-3.5-turbo", 
                         openai_api_key=azure_openai_api_key,
                         engine=deployment_name)

In [27]:


# Fonction pour répondre aux questions
def ask_book_question(question, book_attributes):
    # Concaténer les attributs du livre pour fournir le contexte
    context = "\n".join(book_attributes)
    
    # Créer le message pour le modèle
    messages = [
        HumanMessage(content=f"{context}\n\nQuestion: {question}")
    ]
    
    # Utiliser le modèle pour générer la réponse
    response = chat_model(messages)
    return response.content

# Fonction pour obtenir les informations d'un livre
def get_book_info(book_title):
    # Filtrer les informations du livre correspondant au titre donné
    book_info = df_filtre[df_filtre['Title'].str.contains(book_title, case=False, na=False)]

    # Vérifier si le livre existe dans le DataFrame
    if not book_info.empty:
        # Attributs du livre à utiliser comme contexte
        book_attributes = [
            f"Titre: {book_info.iloc[0]['Title']}",
            f"Auteur: {book_info.iloc[0]['Author']}",
            f"Langue: {book_info.iloc[0]['Language']}",
            f"Sujet: {book_info.iloc[0]['Subject']}",
            f"Résumé: {book_info.iloc[0]['Summary']}",
        ]
        return book_attributes, book_info.iloc[0]['Title'], book_info.iloc[0]['Ebook ID']
    else:
        return None, None, None

# Fonction pour récupérer le texte d'un livre à partir de Gutenberg
def fetch_book_text(ebook_id):
    url = f"https://www.gutenberg.org/ebooks/{ebook_id}.txt.utf-8"  # URL pour le téléchargement en .txt
    response = requests.get(url)
    if response.status_code == 200:
        return response.text
    else:
        return None

# Demander à l'utilisateur de saisir le titre du livre
book_title = input("Veuillez entrer le titre du livre : ")

# Obtenir les attributs du livre
book_attributes, title, ebook_id = get_book_info(book_title)

if book_attributes is not None:
    # Poser une question sur le sujet du livre
    question_subject = f"De quels sujets traite le livre '{title}' ?"
    subjects = ask_book_question(question_subject, book_attributes)
    print(f"Sujets traités dans le livre : {subjects}")

    # Poser une question pour extraire les noms des personnages
    question_characters = f"Quels sont les noms des personnages mentionnés dans le résumé du livre '{title}' ?"
    character_names = ask_book_question(question_characters, book_attributes)
    print(f"Noms des personnages mentionnés dans le résumé : {character_names}")

    # Demander à l'utilisateur s'il souhaite récupérer le texte du livre
    fetch_text = input("Souhaitez-vous récupérer le texte complet du livre ? (oui/non) : ")
    
    if fetch_text.lower() == 'oui':
        book_text = fetch_book_text(ebook_id)
        if book_text:
            print("Texte du livre récupéré avec succès.")
            # Poser une question sur le texte du livre
            question_text = input("Posez votre question sur le texte du livre : ")
            answer_text = ask_book_question(question_text, [book_text])
            print(f"Réponse : {answer_text}")
        else:
            print("Erreur lors de la récupération du texte du livre.")
else:
    print("Aucun livre trouvé avec ce titre.")


Sujets traités dans le livre : "L'Assommoir" traite principalement des sujets suivants :

1. **Pauvreté** : Le roman explore les conditions de vie difficiles de la classe ouvrière à Paris.
2. **Alcoolisme** : Il examine l'impact destructeur de l'alcoolisme sur les individus et les familles.
3. **Lutte domestique** : Les tensions et les conflits au sein de la famille de Gervaise sont au cœur de l'intrigue.
4. **Espoir et désillusion** : Gervaise espère améliorer sa vie, mais ses rêves sont souvent contrecarrés par la réalité sociale.
5. **Amour et frustration** : Les relations complexes et souvent tumultueuses sont un thème récurrent. 

Le livre dépeint la lutte pour la survie dans un environnement social oppressant.
Noms des personnages mentionnés dans le résumé : Les personnages mentionnés dans le résumé sont Gervaise et Lantier.


Évaluer l'agent sur les parties utilisant du RAG
​

In [43]:
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain.schema import Document  

# Initialiser le modèle OpenAI pour la génération de texte (LLM)
llm = OpenAI(model="gpt-3.5-turbo-instruct", openai_api_key=azure_openai_api_key, engine=deployment_name)



C:\Users\antoa\AppData\Roaming\Python\Python311\site-packages\IPython\core\interactiveshell.py:3508: UserWarning: WARNING! engine is not default parameter.
                engine was transferred to model_kwargs.
                Please confirm that engine is what you intended.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [53]:

# Utiliser un modèle compatible pour générer les embeddings
embedding_function = OpenAIEmbeddings(
    openai_api_key=azure_openai_api_key, 
    engine=deployment_name  # Spécifiez le moteur ici
)



In [54]:
# Préparer les documents comme une liste d'objets Document
documents = [
    Document(page_content="\n".join(book_attributes), metadata={"title": title, "author": book_info.iloc[0]['Author']})
]



In [55]:
# Créer une base de données d'indexation avec FAISS pour la récupération
vectorstore = FAISS.from_documents(documents, embedding_function)



InvalidRequestError: The embeddings operation does not work with the specified model, gpt-4o. Please choose different model and try again. You can learn more about which models can be used with each operation here: https://go.microsoft.com/fwlink/?linkid=2197993.

In [ ]:
# Configurer la chaîne RetrievalQA
retriever = vectorstore.as_retriever()
qa_chain = RetrievalQA(llm=llm, retriever=retriever)